# Alachua County restaurant inspection analysis
This will take restaurant inspection data by the state of Florida and format it in a more reader-friendly way for publication in print and online. We'll filter for the most egregious current violations at restaurants in Alachua County.

In [10]:
import pandas as pd
import datetime

This reads in the inspection data for District 5, which includes Alachua County, and adds a missing header row.

In [11]:
insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", header = None)
insp.columns = ["District", "CountyNumber", "CountyName", "LicTypeCode", "LicNumber", "BusinessName", "StreetAddress", "City", "ZIP", "InspectNo", "VisitNo", "InspectClass", "InspectType", "InspectDisp", "InspectDate", "#CriticalVio", "#NoncritVio", "#TotalVio", "#HighVio", "#InterVio", "#BasicVio", "PDA", "FoodVio1", "FoodVio2", "FoodVio3", "FoodVio4", "FoodVio5", "FoodVio6", "FoodVio7", "FoodVio8", "FoodVio9", "FoodVio10", "FoodVio11", "FoodVio12", "FoodVio13", "FoodVio14", "FoodVio15", "FoodVio16", "FoodVio17", "FoodVio18", "FoodVio19", "FoodVio20", "FoodVio21", "FoodVio22", "FoodVio23", "FoodVio24", "FoodVio25", "FoodVio26", "FoodVio27", "FoodVio28", "FoodVio29", "FoodVio30", "FoodVio31", "FoodVio32", "FoodVio33", "FoodVio34", "FoodVio35", "FoodVio36", "FoodVio37", "FoodVio38", "FoodVio39", "FoodVio40", "FoodVio41", "FoodVio42", "FoodVio43", "FoodVio44", "FoodVio45", "FoodVio46", "FoodVio47", "FoodVio48", "FoodVio49", "FoodVio50", "FoodVio51", "FoodVio52", "FoodVio53", "FoodVio54", "FoodVio55", "FoodVio56", "FoodVio57", "FoodVio58", "LicenceID", "VisitID"]

In [12]:
insp.head()

,District,CountyNumber,CountyName,LicTypeCode,LicNumber,BusinessName,StreetAddress,City,ZIP,InspectNo,...,FoodVio51,FoodVio52,FoodVio53,FoodVio54,FoodVio55,FoodVio56,FoodVio57,FoodVio58,LicenceID,VisitID
0,D5,11,Alachua,2015,1100012,DOUBLE ENVELOP 71099,2500 NE 39 AVE,GAINESVILLE,32609,1106616,...,0,0,0,0,0,0,0,0,3713828,6463832
1,D5,11,Alachua,2015,1100012,DOUBLE ENVELOP 71099,2500 NE 39 AVE,GAINESVILLE,32609,1106616,...,0,0,0,0,0,0,0,0,3713828,6267656
2,D5,11,Alachua,2015,1100012,DOUBLE ENVELOP 71099,2500 NE 39 AVE,GAINESVILLE,32609,1106616,...,0,0,0,0,0,0,0,0,3713828,6432746
3,D5,11,Alachua,2015,1100025,PFG,4041 NE 54 AVE,GAINESVILLE,32609,1106611,...,0,0,0,0,0,0,0,0,3713765,6267651
4,D5,11,Alachua,2015,1100037,BEAR ARCHERY 72353,4600 SW 41ST BLVD,GAINESVILLE,32608,1106615,...,0,0,0,0,0,0,0,0,3713820,6267655


This creates a DataFrame named 'alachua', filters out records for Alachua County.

In [13]:
alachua = insp[insp.CountyName == 'Alachua']

Looks like I need to switch data in InspectDate from string to data. How do I do that?

In [14]:
date_date = date_date("InspectDate")
datetime.strptime(date_date, "%Y-%m-%d")

NameError: name 'date_date' is not defined

In [ ]:
 insp.InspectDate > datetime.datetime.now() - pd.to_timedelta("30day")